The end result of this exercise should be a file named acquire.py.

1. Using the code from the lesson as a guide and the REST API from https://python.zgulde.net/api/v1/items as we did in the lesson, create a dataframe named items that has all of the data for items.

2. Do the same thing, but for stores (https://python.zgulde.net/api/v1/stores)

3. Extract the data for sales (https://python.zgulde.net/api/v1/sales). There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

4. Save the data in your files to local csv files so that it will be faster to access in the future.

5. Combine the data from your three separate dataframes into one large dataframe.

6. Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

7. Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the acquire.py file and be able to re-run the functions and get the same data.

In [1]:
import pandas as pd
import requests
import os

In [2]:
domain = 'https://python.zgulde.net'
endpoint = '/api/v1/items'
items = []

url = domain + endpoint

response = requests.get(url)
response

<Response [200]>

In [3]:
data = response.json()

In [4]:
data

{'payload': {'items': [{'item_brand': 'Riceland',
    'item_id': 1,
    'item_name': 'Riceland American Jazmine Rice',
    'item_price': 0.84,
    'item_upc12': '35200264013',
    'item_upc14': '35200264013'},
   {'item_brand': 'Caress',
    'item_id': 2,
    'item_name': 'Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 Ct',
    'item_price': 6.44,
    'item_upc12': '11111065925',
    'item_upc14': '11111065925'},
   {'item_brand': 'Earths Best',
    'item_id': 3,
    'item_name': 'Earths Best Organic Fruit Yogurt Smoothie Mixed Berry',
    'item_price': 2.43,
    'item_upc12': '23923330139',
    'item_upc14': '23923330139'},
   {'item_brand': 'Boars Head',
    'item_id': 4,
    'item_name': 'Boars Head Sliced White American Cheese - 120 Ct',
    'item_price': 3.14,
    'item_upc12': '208528800007',
    'item_upc14': '208528800007'},
   {'item_brand': 'Back To Nature',
    'item_id': 5,
    'item_name': 'Back To Nature Gluten Free White Cheddar Rice Thin Crackers',
    'item_price':

In [5]:
page_n = data['payload']['max_page']

In [6]:
page_n

3

In [7]:
type(data)

dict

In [8]:
data['status']

'ok'

In [9]:
data['payload'].keys()

dict_keys(['items', 'max_page', 'next_page', 'page', 'previous_page'])

In [10]:
data_df = pd.DataFrame(data['payload']['items'])

In [11]:
data_df.shape

(20, 6)

In [12]:
url = domain + data['payload']['next_page']
print('next url:', url)

next url: https://python.zgulde.net/api/v1/items?page=2


In [13]:
response = requests.get(url)
data = response.json()
items.extend(data['payload']['items'])

In [14]:
print('next endpoint', data['payload']['next_page'])
url = domain + data['payload']['next_page']
print('next url:', url)

next endpoint /api/v1/items?page=3
next url: https://python.zgulde.net/api/v1/items?page=3


In [15]:
items_df = pd.DataFrame(items)
items_df.shape
# same shape, but different inclusions.

(20, 6)

In [16]:
# setup
domain = 'https://api.data.codeup.com'
endpoint = '/api/v1/items'
items = []

# For each page -- until next page is None
url = domain + endpoint
response = requests.get(url)
data = response.json()
items.extend(data['payload']['items'])
# update the end point
endpoint = data['payload']['next_page']

In [104]:
def get_items_df(use_cache=True):
    '''
    This function creates a request from the REST API at https://api.data.codeup.com/api/v1/stores
    and transforms the response into a pandas dataframe named items. It then saves the data as a csv file.
    '''
    # If the cached parameter is True, read the csv file on disk in the same folder as this file 
    if os.path.exists('items_df.csv') and use_cache:
        print('Using cached CSV')
        return pd.read_csv('items_df.csv')
   
    # Otherwise, proceed with the following
    print('CSV not present, initializating acquisition')
    
    # create the empty list which will be appended with data with each iteration 
    items = []

    # define the url from where the data is stored
    domain = 'https://api.data.codeup.com'
    endpoint = '/api/v1/items'
    url = domain + endpoint
    
    # define the response by the request
    response = requests.get(url)
    # convert the response to json
    data = response.json()
    # define the number of pages based on the max_page value 
    n = data['payload']['max_page']
    # Create a loop to iterate through each page starting with page 1 and ending on page n + 1
    # be sure to include the last page.
    # p is the page number
    for p in range(1, n+1):
        # define the new url returned for next page
        new_url = url+"?page="+str(p)
        # define the response requested
        response = requests.get(new_url)
        # convert response to json
        data = response.json()
        #create the variable to hold the items returned from the response
        page_items = data['payload']['items']
        # add the items from the page to the items list and continue to iterate through n pages
        page_items = items.extend(data['payload']['items'])
    
        # Create a dataframe of the items_list that now hold all the items from all pages
    items_df = pd.DataFrame(items)
    
    print('iteration complete, creating CSV')
        #also cache the data we read from the REST API to a file on disk
    items_df.to_csv('items_df.csv', index=False)
        

    print('DataFrame available for use')
    
    return items_df

In [105]:
items_df = get_items_df()

CSV not present, initializating acquisition
iteration complete, creating CSV
DataFrame available for use


In [106]:
items_df.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 Ct,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixed Berry,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice Thin Crackers,2.61,759283100036,759283100036


2. Do the same thing, but for stores (https://python.zgulde.net/api/v1/stores)

In [107]:
def get_stores_df(use_cache=True):
    '''
    This function creates a request from the REST API at https://api.data.codeup.com/api/v1/stores
    and transforms the response into a pandas dataframe named items. It then saves the data as a csv file.
    '''
    # If the cached parameter is True, read the csv file on disk in the same folder as this file 
    if os.path.exists('stores_df.csv') and use_cache:
        print('Using cached CSV')
        return pd.read_csv('stores_df.csv')
   
    print('CSV not present, initializating acquisition')
    
    # create the empty list which will be appended with data with each iteration 
    store_list = []
    print('Creating store list')
    # define the url from where the data is stored
    domain = 'https://python.zgulde.net'
    endpoint = '/api/v1/stores'
    url = domain + endpoint

    # define the response by the request
    response = requests.get(url)
    # convert the response to json
    data = response.json()
    # define the number of pages based on the max_page value 
    n = data['payload']['max_page']
    
    print('Iterating through webpages and appending content to store list')
    # Create a loop to iterate through each page starting with page 1 and ending on page n + 1
    # be sure to include the last page.
    # p is the page number
    for p in range(1, n+1):
        # define the new url returned for next page
        new_url = url+"?page="+str(p)
        # define the response requested
        response = requests.get(new_url)
        # convert response to json
        data = response.json()
        #create the variable to hold the items returned from the response
        page_stores = data['payload']['stores']
        # add the items from the page to the items list and continue to iterate through n pages
        page_stores = store_list.extend(data['payload']['stores'])
        
    print('iteration complete, creating CSV')
        # Create a dataframe of the items_list that now hold all the items from all pages
        
    stores_df = pd.DataFrame(store_list)
        
    stores_df.to_csv('stores_df.csv', index=False)
    
    print('DataFrame available for use')
    
    return stores_df

In [108]:
stores_df = get_stores_df()
stores_df

Using cached CSV


,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218
5,1015 S WW White Rd,San Antonio,6,TX,78220
6,12018 Perrin Beitel Rd,San Antonio,7,TX,78217
7,15000 San Pedro Ave,San Antonio,8,TX,78232
8,735 SW Military Dr,San Antonio,9,TX,78221
9,8503 NW Military Hwy,San Antonio,10,TX,78231


In [109]:
stores_df.dtypes

store_address    object
store_city       object
store_id          int64
store_state      object
store_zipcode     int64
dtype: object

In [110]:
# assuming stores is supposed to just be ten rows, moving forward.
def get_sales_df(use_cache=True):
    '''
    This function creates a request from the REST API at https://api.data.codeup.com/api/v1/stores
    and transforms the response into a pandas dataframe named items. It then saves the data as a csv file.
    '''
    # If the cached parameter is True, read the csv file on disk in the same folder as this file 
    if os.path.exists('sales_df.csv') and use_cache:
        print('Using cached CSV')
        return pd.read_csv('sales_df.csv')
   
    print('CSV not present, initializating acquisition')
    
    # create the empty list which will be appended with data with each iteration 
    sales_list = []
    print('Creating sales list')
    
    # define the url from where the data is stored
    domain = 'https://python.zgulde.net'
    endpoint = '/api/v1/sales'
    url = domain + endpoint

    # define the response by the request
    response = requests.get(url)
    # convert the response to json
    data = response.json()
    # define the number of pages based on the max_page value 
    n = data['payload']['max_page']
    
    print('Iterating through webpages and appending content to store list')
    # Create a loop to iterate through each page starting with page 1 and ending on page n + 1
    # be sure to include the last page.
    # p is the page number
    for p in range(1, n+1):
        # define the new url returned for next page
        new_url = url+"?page="+str(p)
        # define the response requested
        response = requests.get(new_url)
        # convert response to json
        data = response.json()
        #create the variable to hold the items returned from the response
        page_sales = data['payload']['sales']
        # add the items from the page to the items list and continue to iterate through n pages
        page_sales = sales_list.extend(data['payload']['sales'])
        
    print('iteration complete, creating CSV')
        # Create a dataframe of the items_list that now hold all the items from all pages
        
    sales_df = pd.DataFrame(sales_list)
    # Noticed some keys need to be fixed
    #sales_df.rename(columns={'item':'item_id','store':'store_id'},inplace=True)
    sales_df = sales_df.rename(columns={'item':'item_id','store':'store_id'})
        
    sales_df.to_csv('sales_df.csv', index=False)
    
    print('DataFrame available for use')
    
    return sales_df

In [111]:
sales_df = get_sales_df()

Using cached CSV


In [65]:
sales_df.dtypes

item_id          int64
sale_amount    float64
sale_date       object
sale_id          int64
store_id         int64
dtype: object

In [56]:
#sales_df.rename(columns={'item':'item_id','store':'store_id'},inplace=True)
#shifted this into function

In [121]:
def get_combined_df(use_cache=True):
    '''
    This function creates a request from the REST API at https://api.data.codeup.com/api/v1/stores
    and transforms the response into a pandas dataframe named items. It then saves the data as a csv file.
    '''
    # If the cached parameter is True, read the csv file on disk in the same folder as this file 
    if os.path.exists('sales_stores_items.csv') and use_cache:
        print('Using cached CSV')
        return pd.read_csv('sales_stores_items.csv')
    
    #Otherwise, create the DF using previous cached CSVs.
    print('Using cached CSVs to created a dataframe')
    
    #hashes can be removed in case of error. Used to check status. 
    sales = get_sales_df()
    #print('SALES')
    #print(sales.shape)
    #print(sales.columns)
    stores = get_stores_df()
    #print('STORES')
    #print(stores.shape)
    #print(stores.columns) 
    #print('ITEMS')
    items = get_items_df()
    #print(items.shape)
    #print(items.columns)
    
    # I went back to my previous function and renamed the columns items and sale to their appropriate ids, so now these can be merged
    print('combining sales and stores on store_id')
    #sales_stores = sales.merge(stores, how='inner', on='store_id')
    cronenberg = sales.merge(stores, on='store_id')
    
    print('combining again on item_id')
    combined_df = cronenberg.merge(items, how='inner', on='item_id')
    
    print('Dataframe ready, sending to a CSV')
    # CSV from CVS
    combined_df.to_csv('sales_stores_items.csv')
    
    return combined_df

In [130]:
combined_df = get_combined_df()

Using cached CSV


In [131]:
combined_df

,sale_id,store_id,item_id,item_brand,item_name,item_price,sale_amount,sale_date,store_address,store_city,store_state,store_zipcode,item_upc12,item_upc14
0,1,1,1,Riceland,Riceland American Jazmine Rice,0.84,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",12125 Alamo Ranch Pkwy,San Antonio,TX,78253,35200264013,35200264013
1,2,1,1,Riceland,Riceland American Jazmine Rice,0.84,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",12125 Alamo Ranch Pkwy,San Antonio,TX,78253,35200264013,35200264013
2,3,1,1,Riceland,Riceland American Jazmine Rice,0.84,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",12125 Alamo Ranch Pkwy,San Antonio,TX,78253,35200264013,35200264013
3,4,1,1,Riceland,Riceland American Jazmine Rice,0.84,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",12125 Alamo Ranch Pkwy,San Antonio,TX,78253,35200264013,35200264013
4,5,1,1,Riceland,Riceland American Jazmine Rice,0.84,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",12125 Alamo Ranch Pkwy,San Antonio,TX,78253,35200264013,35200264013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912995,912996,10,50,Choice,Choice Organic Teas Black Tea Classic Black - 16 Ct,5.20,63.0,"Wed, 27 Dec 2017 00:00:00 GMT",8503 NW Military Hwy,San Antonio,TX,78231,47445919221,47445919221
912996,912997,10,50,Choice,Choice Organic Teas Black Tea Classic Black - 16 Ct,5.20,59.0,"Thu, 28 Dec 2017 00:00:00 GMT",8503 NW Military Hwy,San Antonio,TX,78231,47445919221,47445919221
912997,912998,10,50,Choice,Choice Organic Teas Black Tea Classic Black - 16 Ct,5.20,74.0,"Fri, 29 Dec 2017 00:00:00 GMT",8503 NW Military Hwy,San Antonio,TX,78231,47445919221,47445919221
912998,912999,10,50,Choice,Choice Organic Teas Black Tea Classic Black - 16 Ct,5.20,62.0,"Sat, 30 Dec 2017 00:00:00 GMT",8503 NW Military Hwy,San Antonio,TX,78231,47445919221,47445919221


In [ ]:
from mitosheet import *; # Analysis Name:id-vyazfeqetm;
    
# Deleted column Unnamed: 0 from combined_df
combined_df.drop(['Unnamed: 0'], axis=1, inplace=True)


In [128]:
combined_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [119]:
import mitosheet
mitosheet.sheet(combined_df, analysis_to_replay="id-lqoyfzlmcs")

MitoWidget(analysis_data_json='{"analysisName": "id-lqoyfzlmcs", "analysisToReplay": null, "code": [], "stepSu…

In [117]:

# Deleted column Unnamed: 0 from combined_df
combined_df.drop(['Unnamed: 0'], axis=1, inplace=True)

# Deleted column Unnamed: 0.1 from combined_df
combined_df.drop(['Unnamed: 0.1'], axis=1, inplace=True)

# Reordered item_brand in combined_df
combined_df_columns = [col for col in combined_df.columns if col != 'item_brand']
combined_df_columns.insert(1, 'item_brand')
combined_df = combined_df[combined_df_columns]

# Reordered item_name in combined_df
combined_df_columns = [col for col in combined_df.columns if col != 'item_name']
combined_df_columns.insert(2, 'item_name')
combined_df = combined_df[combined_df_columns]

# Reordered item_price in combined_df
combined_df_columns = [col for col in combined_df.columns if col != 'item_price']
combined_df_columns.insert(3, 'item_price')
combined_df = combined_df[combined_df_columns]

# Reordered sale_id in combined_df
combined_df_columns = [col for col in combined_df.columns if col != 'sale_id']
combined_df_columns.insert(0, 'sale_id')
combined_df = combined_df[combined_df_columns]

# Reordered store_id in combined_df
combined_df_columns = [col for col in combined_df.columns if col != 'store_id']
combined_df_columns.insert(0, 'store_id')
combined_df = combined_df[combined_df_columns]

# Reordered store_id in combined_df
combined_df_columns = [col for col in combined_df.columns if col != 'store_id']
combined_df_columns.insert(2, 'store_id')
combined_df = combined_df[combined_df_columns]

# Reordered item_id in combined_df
combined_df_columns = [col for col in combined_df.columns if col != 'item_id']
combined_df_columns.insert(3, 'item_id')
combined_df = combined_df[combined_df_columns]

# Reordered item_id in combined_df
combined_df_columns = [col for col in combined_df.columns if col != 'item_id']
combined_df_columns.insert(1, 'item_id')
combined_df = combined_df[combined_df_columns]

# Reordered store_id in combined_df
combined_df_columns = [col for col in combined_df.columns if col != 'store_id']
combined_df_columns.insert(0, 'store_id')
combined_df = combined_df[combined_df_columns]

# Reordered store_id in combined_df
combined_df_columns = [col for col in combined_df.columns if col != 'store_id']
combined_df_columns.insert(1, 'store_id')
combined_df = combined_df[combined_df_columns]


In [129]:
combined_df.to_csv('sales_stores_items.csv', index=False)

In [82]:
sales_df.dtypes

item_id          int64
sale_amount    float64
sale_date       object
sale_id          int64
store_id         int64
dtype: object

In [83]:
stores_df.dtypes

store_address    object
store_city       object
store_id          int64
store_state      object
store_zipcode     int64
dtype: object

In [87]:
stores_df.columns

Index(['store_address', 'store_city', 'store_id', 'store_state',
       'store_zipcode'],
      dtype='object')

In [88]:
sales_df.columns

Index(['item_id', 'sale_amount', 'sale_date', 'sale_id', 'store_id'], dtype='object')